<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    76
L    65
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0.0    93
1.0    83
Name: StarStock, dtype: int64

In [7]:
# v40/v40n stocks
mypf['Criteria'].value_counts()

V40N    39
V40     38
2T3Y     8
V200     6
SS       6
TBD      2
Name: Criteria, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1.0    163
0.0     13
Name: LatestQtr, dtype: int64

In [9]:
# star + v40/v40n stocks
mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]['StarStock'].value_counts()

1.0    41
0.0    36
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    141
1     37
Name: InPortfolio, dtype: int64

In [11]:
# star stocks in portfolio
mypf[(mypf['InPortfolio'] == 1)]['StarStock'].value_counts()

1.0    22
0.0    13
Name: StarStock, dtype: int64

In [12]:
# criteria of portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Criteria'].value_counts()

V40     11
V40N     8
2T3Y     8
SS       5
TBD      2
V200     1
Name: Criteria, dtype: int64

In [13]:
# strategies in portfolio stocks
mypf[(mypf['InPortfolio'] == 1)]['Strategy'].value_counts()

ATH     21
2T3Y    12
Name: Strategy, dtype: int64

In [14]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [15]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [16]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0] 
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

In [17]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-12 00:00:00+05:30,18315.0,^NSEI,3.18,0.59,0.31,18812.0,7610.0,4.44,3.0
2023-05-19 00:00:00+05:30,18203.0,^NSEI,2.32,0.54,0.48,18812.0,7610.0,5.44,3.0
2023-05-26 00:00:00+05:30,18499.0,^NSEI,3.79,0.41,0.42,18812.0,7610.0,2.79,2.0
2023-06-02 00:00:00+05:30,18534.0,^NSEI,3.83,0.34,0.84,18812.0,7610.0,2.48,1.0
2023-06-09 00:00:00+05:30,18563.0,^NSEI,3.87,0.27,0.87,18812.0,7610.0,2.22,1.0


In [18]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [19]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [20]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'Criteria', 'Strategy']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 37


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,960.0,-40.18,-4.36,-2.41,98.07,350.0,1354.78,45.2,19.20,M,58.0,1.0,0.0,2T3Y,2T3Y
VALIANTORG.NS,573.0,0.20,0.00,3.16,86.88,223.0,530.15,24.0,18.30,M,24.0,0.0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,306.0,-4.72,0.00,0.33,76.83,216.0,320.64,35.4,48.10,M,-36.0,1.0,0.0,2T3Y,2T3Y
LUXIND.NS,1494.0,-3.92,-1.16,1.74,82.55,204.0,1499.87,21.4,32.70,M,-53.0,1.0,0.0,V40N,2T3Y
BANDHANBNK.NS,257.0,5.14,0.00,2.15,81.90,181.0,252.96,NaN,1000.00,L,NaN,1.0,1.0,2T3Y,2T3Y
NIACL.NS,118.0,12.33,0.95,0.90,83.26,169.0,116.60,NaN,18.60,M,NaN,1.0,0.0,TBD,ATH
AAVAS.NS,1354.0,-27.09,-2.21,-2.07,71.93,136.0,1405.00,NaN,24.90,M,NaN,1.0,1.0,2T3Y,2T3Y
TEAMLEASE.NS,2314.0,-10.27,-2.02,-0.41,78.06,128.0,2239.86,NaN,34.80,M,NaN,1.0,1.0,V40N,2T3Y
INDIGOPNTS.NS,1417.0,8.25,-0.31,3.28,79.65,120.0,1408.25,104.9,51.10,M,51.0,1.0,1.0,V40N,2T3Y


In [21]:
# top 10 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
GLAND.NS,960.0,-40.18,-4.36,-2.41,98.07,350.0,1354.78,45.2,19.2,M,58.0,1.0,0.0,2T3Y,2T3Y
VALIANTORG.NS,573.0,0.20,0.00,3.16,86.88,223.0,530.15,24.0,18.3,M,24.0,0.0,0.0,2T3Y,2T3Y
VAIBHAVGBL.NS,306.0,-4.72,0.00,0.33,76.83,216.0,320.64,35.4,48.1,M,-36.0,1.0,0.0,2T3Y,2T3Y
LUXIND.NS,1494.0,-3.92,-1.16,1.74,82.55,204.0,1499.87,21.4,32.7,M,-53.0,1.0,0.0,V40N,2T3Y
BANDHANBNK.NS,257.0,5.14,0.00,2.15,81.90,181.0,252.96,NaN,1000.0,L,NaN,1.0,1.0,2T3Y,2T3Y
NIACL.NS,118.0,12.33,0.95,0.90,83.26,169.0,116.60,NaN,18.6,M,NaN,1.0,0.0,TBD,ATH
AAVAS.NS,1354.0,-27.09,-2.21,-2.07,71.93,136.0,1405.00,NaN,24.9,M,NaN,1.0,1.0,2T3Y,2T3Y


In [22]:
# top 5 from portfolio to book profit
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ASIANPAINT.NS,3181.0,4.80,-0.43,1.28,14.77,11.0,3048.30,87.9,73.8,H,16.0,1.0,1.0,V40,ATH
PIDILITIND.NS,2615.0,2.75,-0.04,1.05,14.32,11.0,2442.09,94.6,104.0,H,-10.0,1.0,1.0,V40,ATH
BAJFINANCE.NS,7037.0,6.82,-0.23,2.09,14.21,12.0,6365.23,52.6,37.0,H,30.0,1.0,1.0,V40,ATH
RELIANCE.NS,2482.0,0.65,-0.24,1.04,16.92,13.0,2464.49,29.1,25.2,H,13.0,1.0,1.0,V40,ATH
AMBUJACEM.NS,456.0,-0.72,0.44,1.99,28.51,29.0,408.85,23.9,32.3,L,-35.0,0.0,0.0,TBD,ATH
BAJAJFINSV.NS,1466.0,-2.69,-0.40,1.45,29.44,30.0,1449.21,39.2,36.4,M,7.0,1.0,1.0,V40,NaN
ICICIGI.NS,1216.0,4.97,-0.09,1.35,42.84,33.0,1106.17,46.2,1000.0,M,-2065.0,1.0,1.0,V40,ATH


In [23]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [24]:
# star + v40/v40n stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 26)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 8


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
SUNTV.NS,444.0,-5.43,-0.21,0.46,802.0,249.0,64.74,81.0,NaN,NaN,M,1.0,1.0,V40N,NaN
RAJESHEXPO.NS,571.0,-14.84,-0.30,-0.86,949.0,440.0,74.26,66.0,NaN,NaN,M,1.0,1.0,V40N,NaN
DIXON.NS,4074.0,11.69,0.05,3.44,5750.0,374.0,31.18,41.0,NaN,NaN,M,1.0,1.0,V40N,NaN
ASTRAZEN.NS,3478.0,6.08,0.55,0.43,4650.0,1290.0,34.88,34.0,NaN,NaN,M,1.0,1.0,V40N,NaN
BERGEPAINT.NS,648.0,7.24,-0.17,0.98,858.0,260.0,35.12,32.0,1.0,0.0,M,1.0,1.0,V40,NaN
ANGELONE.NS,1489.0,12.58,0.45,2.69,1894.0,208.0,24.02,27.0,NaN,NaN,M,1.0,1.0,V40N,NaN
OFSS.NS,3541.0,17.52,0.73,1.66,4392.0,1232.0,26.93,24.0,1.0,0.0,M,1.0,1.0,V40N,NaN
TCS.NS,3209.0,-0.46,-0.03,0.37,3886.0,1550.0,28.98,21.0,1.0,0.0,H,1.0,1.0,V40,NaN


In [25]:
# top 5 from star + v40/v40n for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
RAJESHEXPO.NS,571.0,-14.84,-0.30,-0.86,949.0,440.0,74.26,66.0,NaN,NaN,M,1.0,1.0,V40N,NaN
SUNTV.NS,444.0,-5.43,-0.21,0.46,802.0,249.0,64.74,81.0,NaN,NaN,M,1.0,1.0,V40N,NaN
TCS.NS,3209.0,-0.46,-0.03,0.37,3886.0,1550.0,28.98,21.0,1.0,0.0,H,1.0,1.0,V40,NaN
ASTRAZEN.NS,3478.0,6.08,0.55,0.43,4650.0,1290.0,34.88,34.0,NaN,NaN,M,1.0,1.0,V40N,NaN
BERGEPAINT.NS,648.0,7.24,-0.17,0.98,858.0,260.0,35.12,32.0,1.0,0.0,M,1.0,1.0,V40,NaN


In [26]:
# star + v40/v40n stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 7


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
HINDUNILVR.NS,2636.0,2.50,0.12,0.78,2728.0,1406.0,6.96,3.0,1.0,1.0,H,1.0,1.0,V40,NaN
ITC.NS,438.0,22.13,1.95,1.68,450.0,124.0,3.68,3.0,1.0,1.0,H,1.0,1.0,V40,NaN
ICICIBANK.NS,938.0,5.16,0.45,0.98,954.0,252.0,2.28,2.0,1.0,1.0,H,1.0,1.0,V40,NaN
BAJAJ-AUTO.NS,4743.0,21.69,0.85,2.00,4801.0,1803.0,1.93,1.0,NaN,NaN,L,1.0,1.0,V40,NaN
CERA.NS,7887.0,33.95,2.43,2.25,7978.0,2078.0,1.54,1.0,1.0,1.0,H,1.0,1.0,V40N,NaN
POLYCAB.NS,3548.0,24.34,1.93,2.15,3595.0,526.0,1.53,1.0,1.0,0.0,M,1.0,1.0,V40N,NaN
TITAN.NS,2876.0,11.44,0.77,1.41,2907.0,733.0,1.43,1.0,1.0,1.0,H,1.0,1.0,V40,NaN


In [27]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Criteria'] != 'V40') & (tmp_df['Criteria'] != 'V40N')]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ADANITRANS.NS,831.0,-61.82,-6.16,-2.38,4105.0,143.0,82.64,394.0,1.0,-1.0,L,1.0,1.0,NaN,NaN
ADANIGREEN.NS,964.0,-37.87,-3.87,0.43,2970.0,25.0,68.12,208.0,-1.0,-1.0,L,1.0,1.0,NaN,NaN
ADANIENT.NS,2451.0,-14.62,-0.66,3.14,4165.0,66.0,41.82,70.0,1.0,0.0,M,1.0,1.0,NaN,NaN
EMAMILTD.NS,383.0,-9.52,-0.71,0.80,588.0,144.0,46.17,54.0,-1.0,1.0,L,1.0,1.0,V200,NaN
LTIM.NS,4823.0,4.89,0.09,0.49,7439.0,1243.0,42.22,54.0,1.0,0.0,M,1.0,1.0,V200,NaN
GODREJIND.NS,465.0,5.21,0.23,1.33,644.0,235.0,43.77,38.0,0.0,1.0,L,1.0,1.0,NaN,NaN
TATAPOWER.NS,221.0,4.34,0.00,0.99,285.0,26.0,24.71,29.0,1.0,0.0,M,1.0,1.0,NaN,NaN
INDIANB.NS,285.0,7.66,1.89,0.34,356.0,40.0,22.47,25.0,1.0,1.0,H,1.0,1.0,NaN,NaN
IGL.NS,463.0,8.95,0.94,0.84,563.0,207.0,28.09,22.0,NaN,NaN,L,1.0,1.0,NaN,NaN


In [28]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
BRITANNIA.NS,4879.0,16.91,1.37,1.38,4893.0,1930.0,0.47,0.0,1.0,1.0,H,1.0,1.0,NaN,NaN
CERA.NS,7887.0,33.95,2.43,2.25,7978.0,2078.0,1.54,1.0,1.0,1.0,H,1.0,1.0,V40N,NaN
LT.NS,2363.0,12.20,1.00,0.53,2384.0,672.0,1.23,1.0,1.0,0.0,M,1.0,1.0,V200,NaN
POLYCAB.NS,3548.0,24.34,1.93,2.15,3595.0,526.0,1.53,1.0,1.0,0.0,M,1.0,1.0,V40N,NaN
TITAN.NS,2876.0,11.44,0.77,1.41,2907.0,733.0,1.43,1.0,1.0,1.0,H,1.0,1.0,V40,NaN
BHARTIARTL.NS,834.0,5.51,0.89,1.01,851.0,258.0,2.87,2.0,1.0,1.0,H,1.0,1.0,NaN,NaN
ICICIBANK.NS,938.0,5.16,0.45,0.98,954.0,252.0,2.28,2.0,1.0,1.0,H,1.0,1.0,V40,NaN
HINDUNILVR.NS,2636.0,2.50,0.12,0.78,2728.0,1406.0,6.96,3.0,1.0,1.0,H,1.0,1.0,V40,NaN
ITC.NS,438.0,22.13,1.95,1.68,450.0,124.0,3.68,3.0,1.0,1.0,H,1.0,1.0,V40,NaN


In [29]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
ACC.NS,1837.0,-14.23,-0.98,0.68,2748.0,863.0,48.33,50.0,0.0,0.0,L,0.0,0.0,NaN,NaN
PVRINOX.NS,1414.0,-12.80,-0.92,-0.89,1917.0,1348.0,88.40,36.0,1.0,-1.0,L,0.0,0.0,NaN,NaN
PGHH.NS,13851.0,-0.23,-0.24,0.02,15778.0,8145.0,25.25,14.0,0.0,0.0,L,0.0,0.0,V40,NaN
SIEMENS.NS,3659.0,18.51,1.30,0.86,3891.0,834.0,7.59,6.0,1.0,1.0,H,0.0,0.0,NaN,NaN
NESTLEIND.NS,22073.0,11.89,0.62,1.52,22368.0,8621.0,2.15,1.0,1.0,1.0,H,0.0,0.0,V40,NaN


In [30]:
# stocks prospects 

excluded_stocks = ['BANDHANBNK.NS','GICRE.NS','JMFINANCIL.NS','NIACL.NS','PNB.NS','UJJIVANSFB.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 0


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,


In [31]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,157.0,-10.14,-1.14,0.68,303.0,105.0,73.74,93.0,1.0,1.0,H,1.0,0.0,NaN,NaN


In [32]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,157.0,-10.14,-1.14,0.68,303.0,105.0,73.74,93.0,1.0,1.0,H,1.0,0.0,NaN,NaN


In [33]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 18


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
BRITANNIA.NS,4879.0,16.91,1.37,1.38,4893.0,1930.0,0.47,0.0,1.0,1.0,H,1.0,1.0,NaN,NaN
TITAN.NS,2876.0,11.44,0.77,1.41,2907.0,733.0,1.43,1.0,1.0,1.0,H,1.0,1.0,V40,NaN
CERA.NS,7887.0,33.95,2.43,2.25,7978.0,2078.0,1.54,1.0,1.0,1.0,H,1.0,1.0,V40N,NaN
MARUTI.NS,9628.0,9.43,0.35,1.58,9736.0,3925.0,1.86,1.0,1.0,1.0,H,1.0,0.0,NaN,NaN
NESTLEIND.NS,22073.0,11.89,0.62,1.52,22368.0,8621.0,2.15,1.0,1.0,1.0,H,0.0,0.0,V40,NaN
ICICIBANK.NS,938.0,5.16,0.45,0.98,954.0,252.0,2.28,2.0,1.0,1.0,H,1.0,1.0,V40,NaN
BHARTIARTL.NS,834.0,5.51,0.89,1.01,851.0,258.0,2.87,2.0,1.0,1.0,H,1.0,1.0,NaN,NaN
M&M.NS,1371.0,7.76,0.31,1.84,1411.0,262.0,3.48,3.0,1.0,1.0,H,1.0,1.0,NaN,NaN
ITC.NS,438.0,22.13,1.95,1.68,450.0,124.0,3.68,3.0,1.0,1.0,H,1.0,1.0,V40,NaN


In [34]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 13


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
EICHERMOT.NS,3581.0,6.49,0.71,2.19,3850.0,1252.0,10.35,8.0,1.0,1.0,H,1.0,1.0,V40N,ATH
M&M.NS,1371.0,7.76,0.31,1.84,1411.0,262.0,3.48,3.0,1.0,1.0,H,1.0,1.0,NaN,NaN
HDFCLIFE.NS,582.0,6.54,0.37,1.65,757.0,340.0,41.97,30.0,1.0,1.0,H,1.0,0.0,V40,NaN
MARUTI.NS,9628.0,9.43,0.35,1.58,9736.0,3925.0,1.86,1.0,1.0,1.0,H,1.0,0.0,NaN,NaN
NESTLEIND.NS,22073.0,11.89,0.62,1.52,22368.0,8621.0,2.15,1.0,1.0,1.0,H,0.0,0.0,V40,NaN
TATACONSUM.NS,801.0,5.81,0.13,1.46,866.0,172.0,9.37,8.0,1.0,1.0,H,1.0,1.0,SS,ATH
TITAN.NS,2876.0,11.44,0.77,1.41,2907.0,733.0,1.43,1.0,1.0,1.0,H,1.0,1.0,V40,NaN
BRITANNIA.NS,4879.0,16.91,1.37,1.38,4893.0,1930.0,0.47,0.0,1.0,1.0,H,1.0,1.0,NaN,NaN
HAVELLS.NS,1346.0,9.51,0.08,1.28,1456.0,451.0,10.95,8.0,1.0,1.0,H,1.0,0.0,V40,NaN


In [35]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
INDUSTOWER.NS,157.0,-10.14,-1.14,0.68,303.0,105.0,73.74,93.0,1.0,1.0,H,1.0,0.0,NaN,NaN


In [36]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 43


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
YESBANK.NS,17.0,-2.35,0.00,0.00,383.0,11.0,98.39,2153.0,0.0,0.0,L,1.0,0.0,NaN,NaN
ADANITRANS.NS,831.0,-61.82,-6.16,-2.38,4105.0,143.0,82.64,394.0,1.0,-1.0,L,1.0,1.0,NaN,NaN
PEL.NS,784.0,-8.37,-6.19,1.63,2950.0,597.0,92.05,276.0,-1.0,0.0,L,1.0,0.0,NaN,NaN
JCHAC.NS,1041.0,-13.28,-1.92,-0.64,3219.0,1017.0,98.91,209.0,NaN,NaN,M,1.0,0.0,V40N,NaN
ADANIGREEN.NS,964.0,-37.87,-3.87,0.43,2970.0,25.0,68.12,208.0,-1.0,-1.0,L,1.0,1.0,NaN,NaN
IBREALEST.NS,65.0,-7.89,-1.41,1.64,191.0,39.0,82.89,194.0,-1.0,-1.0,L,1.0,0.0,NaN,NaN
ZEEL.NS,194.0,-16.00,-1.30,-0.51,544.0,116.0,81.78,180.0,-1.0,1.0,L,1.0,0.0,NaN,NaN
LALPATHLAB.NS,1956.0,-10.90,-0.96,1.03,4186.0,807.0,66.00,114.0,NaN,NaN,M,1.0,0.0,V40N,NaN
BIOCON.NS,241.0,-5.82,-1.56,1.71,483.0,199.0,85.21,100.0,1.0,0.0,M,1.0,0.0,NaN,NaN


In [37]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,Criteria,Strategy
Stock,,,,,,,,,,,,,,,
PVRINOX.NS,1414.0,-12.8,-0.92,-0.89,1917.0,1348.0,88.4,36.0,1.0,-1.0,L,0.0,0.0,NaN,NaN


In [38]:
# save the output
df_prec_dev = df_prec_dev.reset_index()
df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [39]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-05-12 00:00:00+05:30,792.0,BHARTIARTL.NS,1.92,0.77,0.26,851.0,258.0,9.95,7.0
2023-05-19 00:00:00+05:30,806.0,BHARTIARTL.NS,3.30,0.77,0.52,851.0,258.0,7.59,6.0
2023-05-26 00:00:00+05:30,818.0,BHARTIARTL.NS,4.46,0.77,0.52,851.0,258.0,5.56,4.0
2023-06-02 00:00:00+05:30,836.0,BHARTIARTL.NS,6.29,0.89,0.89,851.0,258.0,2.53,2.0
2023-06-09 00:00:00+05:30,834.0,BHARTIARTL.NS,5.51,0.89,1.01,851.0,258.0,2.87,2.0
